# Simple Search Engine
Elasticsearch와 mysql을 결합한 검색엔진을 만들어보겠습니다.
____

In [1]:
import pandas as pd
df = pd.read_csv("movie_doc.csv",sep="\t")

In [2]:
df.head()

,id,title,story,gender,link,image
0,0,인비저블 게스트,"의문의 습격으로 살해 당한 ‘로라’ ‘아드리안’은 연인의 죽음에 절망하고, 범인은 ...",N,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20170828_179/1...
1,1,"나, 다니엘 블레이크",평생을 성실하게 목수로 살아가던 다니엘은 지병인 심장병이 악화되어 일을 계속 해나갈...,F,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20161117_246/1...
2,2,국가부도의 날,"1997년, 대한민국 최고의 경제 호황을 믿어 의심치 않았던 그때, 곧 엄청난 경제...",M,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20181105_232/1...
3,3,당갈,전직 레슬링 선수였던 ‘마하비르 싱 포갓(아미르 칸)’은 아버지의 반대로 금메달의 ...,F,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20180329_2/152...
4,4,스파이더맨: 파 프롬 홈,"‘엔드게임’ 이후 변화된 세상, 스파이더맨 ‘피터 파커’는 학교 친구들과 유럽 여행...",M,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20190527_181/1...


## 2. MySQL에 데이터 적재

In [3]:
import mysql.connector


HOST = "localhost"
USER = "root"
PASSWORD = "Boost111@"
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD,
  database="movie_db"
)

mycursor = mydb.cursor()


In [4]:
sql = "SELECT * FROM movie"
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)
    break

{'id': '0', 'story': '의문의 습격으로 살해 당한 ‘로라’ ‘아드리안’은 연인의 죽음에 절망하고, 범인은 흔적도 없이 사라졌다. 유력한 용의자로 누명을 쓴 ‘아드리안’은 승률 100%의 변호사 ‘버지니아’를 선임한다. 그리고 자신의 무죄를 입증하기 위해 고군분투하던 중 과거 그와 ‘로라’가 은폐한 교통사고와 숨겨진 연관성을 찾게 되는데… 남은 시간은 단 3시간, 사건을 재구성해 무죄를 입증해야 한다!', 'title': '인비저블 게스트', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=159516', 'image': 'https://movie-phinf.pstatic.net/20170828_179/1503887362732snsIK_JPEG/movie_image.jpg?type=f67'}


## 3. ElasticSearch에 데이터 인덱싱

In [7]:
from elasticsearch import Elasticsearch, helpers


In [8]:
import pprint  
INDEX_NAME = "movie_index"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ]
          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "story" : {
          "type" : "text",
          "analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean"
        }
      }

  }
}


In [9]:


try:
    es.transport.close()
except:
    pass
es = Elasticsearch()



In [10]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-10-b31c6028f9a8>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
<ipython-input-10-b31c6028f9a8>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NA

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movie_index'}

In [11]:
movies = [
    {
        "_index": INDEX_NAME,
        "_id" : doc['id'],
        "_source": {
            "title": doc['title'],
            "story": doc['story'],
        }
    }
    for doc in myresult
]

movies[:10]

[{'_index': 'movie_index',
  '_id': '0',
  '_source': {'title': '인비저블 게스트',
   'story': '의문의 습격으로 살해 당한 ‘로라’ ‘아드리안’은 연인의 죽음에 절망하고, 범인은 흔적도 없이 사라졌다. 유력한 용의자로 누명을 쓴 ‘아드리안’은 승률 100%의 변호사 ‘버지니아’를 선임한다. 그리고 자신의 무죄를 입증하기 위해 고군분투하던 중 과거 그와 ‘로라’가 은폐한 교통사고와 숨겨진 연관성을 찾게 되는데… 남은 시간은 단 3시간, 사건을 재구성해 무죄를 입증해야 한다!'}},
 {'_index': 'movie_index',
  '_id': '1',
  '_source': {'title': '나, 다니엘 블레이크',
   'story': '평생을 성실하게 목수로 살아가던 다니엘은 지병인 심장병이 악화되어 일을 계속 해나갈 수 없는 상황이 된다. 다니엘은 실업급여를 받기 위해 찾아간 관공서에서 복잡하고 관료적인 절차 때문에 번번히 좌절한다. 그러던 어느 날 다니엘은 두 아이와 함께 런던에서 이주한 싱글맘 케이티를 만나 도움을 주게되고, 서로를 의지하게 되는데...'}},
 {'_index': 'movie_index',
  '_id': '2',
  '_source': {'title': '국가부도의 날',
   'story': '1997년, 대한민국 최고의 경제 호황을 믿어 의심치 않았던 그때, 곧 엄청난 경제 위기가 닥칠 것을 예견한 한국은행 통화정책팀장 ‘한시현’(김혜수)은 이 사실을 보고하고, 정부는 뒤늦게 국가부도 사태를 막기 위한 비공개 대책팀을 꾸린다. 한편, 곳곳에서 감지되는 위기의 시그널을 포착하고 과감히 사표를 던진 금융맨 ‘윤정학’(유아인)은 국가부도의 위기에 투자하는 역베팅을 결심, 투자자들을 모으기 시작한다. 이런 상황을 알 리 없는 작은 공장의 사장이자 평범한 가장 ‘갑수’(허준호)는 대형 백화점과의 어음 거래 계약서에 도장을 찍고 소박한 행복을 꿈꾼다. 국가부도까지 남은 시간 

In [12]:
try:
    response = helpers.bulk(es, movies)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)


RESPONSE: (403, [])


In [13]:
doc = es.get(index=INDEX_NAME, id=1)
pprint.pprint(doc)

{'_id': '1',
 '_index': 'movie_index',
 '_primary_term': 1,
 '_seq_no': 1,
 '_source': {'story': '평생을 성실하게 목수로 살아가던 다니엘은 지병인 심장병이 악화되어 일을 계속 해나갈 수 없는 상황이 '
                      '된다. 다니엘은 실업급여를 받기 위해 찾아간 관공서에서 복잡하고 관료적인 절차 때문에 번번히 '
                      '좌절한다. 그러던 어느 날 다니엘은 두 아이와 함께 런던에서 이주한 싱글맘 케이티를 만나 도움을 '
                      '주게되고, 서로를 의지하게 되는데...',
             'title': '나, 다니엘 블레이크'},
 '_type': '_doc',
 '_version': 1,
 'found': True}


## 4. ElasticSearch에서 검색후, 검색결과를 MySQL에서 재 검색

In [14]:
query="은퇴한 킬러"
res = es.search(index=INDEX_NAME, q=query)
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))
    print("Title: {}".format(hit['_source']['title']))
    print("줄거리: {}".format(hit['_source']['story']))


Doc ID: '67'  Score: 12.94
Title: 존 윅 - 리로드
줄거리: 업계 최고의 레전드 킬러 ‘존 윅’은 과거를 뒤로한 채 은퇴를 선언하지만, 과거 자신의 목숨을 구했던 옛 동료와 피로 맺은 암살자들의 룰에 의해 로마로 향한다. ‘국제 암살자 연합’을 탈취하려는 옛 동료의 계획으로 ‘존 윅’은 함정에 빠지게 되고, 전세계 암살자들의 총구는 그를 향하는데...
Doc ID: '301'  Score:  7.92
Title: 블레이드 러너
줄거리: 핵전쟁 이후 혼돈으로 무질서로 휩싸인 2019년, 복제인간 ‘로이’를 포함한 ‘넥서스 6’이 오프월드에서 반란을 일으킨 후 지구로 잠입한다. 은퇴한 블레이드 러너였던 '데커드'(해리슨 포드)는 지구에 잠입한 복제 인간들을 찾는 임무와 함께 강제로 복직하게 되고, 탐문 수사를 위해 찾아간 넥서스 6 제조사인 타이렐 사에서 자신이 복제 인간임을 모르는 ‘레이첼’(숀 영)을 마주하게 된다. 한편, 증거의 꼬리를 잡아 수사하던 도중 ‘데커드’는 ‘레이첼’ 덕분에 위기 속에서 목숨을 구원받게 되고, 복제 인간과의 마지막 전투를 앞두게 된다.
Doc ID: '181'  Score:  7.65
Title: 수상한 그녀
줄거리: 아들 자랑이 유일한 낙인 욕쟁이 칠순 할매 오말순(나문희分)은 어느 날, 가족들이 자신을 요양원으로 독립(?)시키려 한다는 청천벽력 같은 사실을 알게 된다. 뒤숭숭한 마음을 안고 밤길을 방황하던 할매 말순은 오묘한 불빛에 이끌려 ‘청춘 사진관’으로 들어간다. 난생 처음 곱게 꽃단장을 하고 영정사진을 찍고 나오는 길, 그녀는 버스 차창 밖에 비친 자신의 얼굴을 보고 경악을 금치 못한다. 오드리 헵번처럼 뽀얀 피부, 날렵한 몸매... 주름진 할매에서 탱탱한 꽃처녀의 몸으로 돌아간 것! 아무도 알아보지 못하는 자신의 젊은 모습에 그녀는 스무살 ‘오두리’가 되어 빛나는 전성기를 즐겨 보기로 마음 먹는데... 2014년 새해, 대한민국에 웃음 보따리를 안겨줄 <수상한 그녀>가 온다!
Doc ID: '

In [15]:
movie_id = res['hits']['hits'][0]["_id"]

In [16]:
sql = "SELECT * FROM movie WHERE id = {}".format(movie_id)
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)

{'id': '67', 'story': '업계 최고의 레전드 킬러 ‘존 윅’은 과거를 뒤로한 채 은퇴를 선언하지만, 과거 자신의 목숨을 구했던 옛 동료와 피로 맺은 암살자들의 룰에 의해 로마로 향한다. ‘국제 암살자 연합’을 탈취하려는 옛 동료의 계획으로 ‘존 윅’은 함정에 빠지게 되고, 전세계 암살자들의 총구는 그를 향하는데...', 'title': '존 윅 - 리로드', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=143932', 'image': 'https://movie-phinf.pstatic.net/20200807_193/1596789703698oLgpX_JPEG/movie_image.jpg?type=f67'}


In [ ]:
#쿼리---> 엘라스틱서치 --> MySQL